In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
# import boto3
from datetime import datetime

In [1]:
from pyspark.sql import SparkSession

# Define the package coordinates
mysql_jar_package = "com.mysql:mysql-connector-j:8.0.33"

spark = SparkSession.builder \
    .appName("olist_order_payments_dataset") \
    .config("spark.cores.max", "1") \
    .config("spark.executor.memory", "2g") \
    .config("spark.jars.packages", mysql_jar_package) \
    .getOrCreate()


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.mysql#mysql-connector-j added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7c23d95c-db47-467b-85fd-b1152d992374;1.0
	confs: [default]
	found com.mysql#mysql-connector-j;8.0.33 in central
	found com.google.protobuf#protobuf-java;3.21.9 in central
:: resolution report :: resolve 175ms :: artifacts dl 7ms
	:: modules in use:
	com.google.protobuf#protobuf-java;3.21.9 from central in [default]
	com.mysql#mysql-connector-j;8.0.33 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------------

In [3]:
file = '../dataset/olist_order_payments_dataset.csv'
df = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'false') \
    .option("escape", '"') \
    .option("quote", '"') \
    .option("multiLine", "true") \
    .option("ignoreLeadingWhiteSpace", "true") \
    .option("ignoreTrailingWhiteSpace", "true") \
    .load(file)
# df.printSchema()
# df.show(5)

In [4]:
# MySQL connection properties
jdbc_url = "jdbc:mysql://mariadb:3306/database_raw"
db_properties = {
    "user": "admin",
    "password": "admin",
    "driver": "com.mysql.cj.jdbc.Driver" # Use "com.mysql.jdbc.Driver" for older connectors
}
table_name = "olist_order_payments_dataset"

# Write the DataFrame to the MySQL table
df.write.jdbc(url=jdbc_url, table=table_name, mode="append", properties=db_properties)

In [5]:
spark.stop()